In [ ]:
keep 
atr_14  , OBV

drop

'ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal', 'trending_market'

In [ ]:
drop_tcn = [
    'open', 'high', 'low', 'typical_price', 'EMA_21', 'SMA_20',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower',
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'trending_market', 'above_sma50', 'ema7_above_ema21',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold',
    'above_sma20', 'macd_positive', 'volume_breakout', 'volume_breakdown',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6','ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal',
]

In [ ]:
"""
tcn_optuna_search.py  —  Robust TCN hyper-parameter optimisation
─────────────────────────────────────────────────────────────────
• Objective  : weighted-F1 with α = 2  (precision has double weight)
• Output     : scaler, best-params (.json), all trials (.csv), history plot
• Tested on  : TensorFlow 2.16 · tcn 3.5 · Optuna 3.x  (CPU & single GPU)
"""

# ───────────────────────── imports ──────────────────────────
import os, json, gc, warnings, optuna
from datetime import datetime
from pathlib import Path
from typing import Dict, Tuple

import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score

import tensorflow as tf
from tensorflow import keras
from tcn import TCN                       #  pip install tcn==3.*
from optuna_integration.tfkeras import TFKerasPruningCallback

# ───────────────────── runtime hygiene ──────────────────────
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

SEED, VAL_FRAC, ALPHA = 42, 0.20, 2.0
N_TRIALS, TIMEOUT = 40, 80 * 60  # seconds

np.random.seed(SEED)
tf.random.set_seed(SEED)
for g in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

# ──────────────────────── paths, drops ─────────────────────
CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
SCALER_PKL = "tcn_scaler.pkl"

DROP_COLS = [
    'open', 'high', 'low', 'typical_price', 'EMA_21', 'SMA_20',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower',
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'trending_market', 'above_sma50', 'ema7_above_ema21',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold',
    'above_sma20', 'macd_positive', 'volume_breakout', 'volume_breakdown',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6','ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal',
]

# ───────────────────────── data load ───────────────────────
print("📊 Loading data …")
df = (
    pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
      .loc["2018-01-01":]
      .drop(columns=[c for c in DROP_COLS if c in df.columns])
      .dropna(subset=["target"])
      .dropna()
)
print(f"Shape  : {df.shape}")
print("Classes:\n", df["target"].value_counts(), "\n")

X_raw, y_raw = df.drop(columns="target").values, df["target"].astype(int).values
n_features   = X_raw.shape[1]

split = int(len(df) * (1 - VAL_FRAC))
scaler = StandardScaler().fit(X_raw[:split])
joblib.dump(scaler, SCALER_PKL)

X_tr_raw = scaler.transform(X_raw[:split]).astype(np.float32)
X_va_raw = scaler.transform(X_raw[split:]).astype(np.float32)
y_tr_raw, y_va_raw = y_raw[:split], y_raw[split:]

pos_rate = y_tr_raw.mean()
CLASS_W0 = np.float32(1.0)
CLASS_W1 = np.float32((1 - pos_rate) / pos_rate if pos_rate else 1.0)
print(f"Class-weights 0 / 1 → {CLASS_W0:.2f} / {CLASS_W1:.2f}")

# ─────────────────── window cache helper ───────────────────
_CACHE: Dict[Tuple[int, int, int], Tuple[np.ndarray, np.ndarray]] = {}

def make_windows(arr: np.ndarray, labels: np.ndarray, win: int):
    k = (id(arr), len(labels), win)
    if k in _CACHE:
        return _CACHE[k]

    Xs, ys = [], []
    for i in range(win, len(arr)):
        Xs.append(arr[i - win:i])
        ys.append(labels[i])
    Xw, yw = np.asarray(Xs, np.float32), np.asarray(ys, np.int8)

    if Xw.nbytes + yw.nbytes < 1_000_000_000:
        _CACHE[k] = (Xw, yw)
    return Xw, yw

# ─────────────── metric: weighted-F1 (α=2) ────────────────
def f1_alpha2(y_true, y_prob) -> float:
    y_pred = (y_prob >= 0.5).astype(int)
    p, r   = precision_score(y_true, y_pred, zero_division=0), \
             recall_score   (y_true, y_pred, zero_division=0)
    return 0. if p+r == 0 else (1+ALPHA)*p*r / (ALPHA*p + r)

# ──────────────── TCN model factory ───────────────────────
def build_tcn(cfg: dict) -> keras.Model:
    dilations = [2 ** i for i in range(cfg["nb_stacks"] * cfg["blocks_per_stack"])]

    inp = keras.layers.Input(shape=(cfg["window"], n_features))
    x   = TCN(
            nb_filters       = cfg["filters"],
            kernel_size      = cfg["kernel"],
            nb_stacks        = cfg["nb_stacks"],
            dilations        = dilations,
            padding          = "causal",
            dropout_rate     = cfg["dropout"],
            activation       = cfg["act"],
            use_skip_connections = True,
            use_batch_norm   = cfg["norm"] == "batch",
            use_layer_norm   = cfg["norm"] == "layer",
            return_sequences = False
         )(inp)

    x   = keras.layers.Dense(cfg["dense"], activation=cfg["act"])(x)
    x   = keras.layers.Dropout(cfg["dropout"])(x)
    out = keras.layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inp, out)

    # balanced BCE
    def weighted_bce(y_t, y_p):
        y_t = tf.cast(y_t, y_p.dtype)
        w   = tf.where(tf.equal(y_t, 1), CLASS_W1, CLASS_W0)
        w   = tf.cast(w, y_p.dtype)
        return tf.reduce_mean(w * keras.losses.binary_crossentropy(y_t, y_p))

    model.compile(keras.optimizers.Adam(cfg["lr"]), loss=weighted_bce)
    return model

# ──────────────────── Optuna objective ────────────────────
def objective(trial: optuna.trial.Trial):
    cfg = dict(
        window           = trial.suggest_int("window", 12, 60, step=6),
        filters          = trial.suggest_categorical("filters", [32, 48, 64, 96]),
        kernel           = trial.suggest_int("kernel", 2, 6),
        nb_stacks        = trial.suggest_int("nb_stacks", 1, 3),
        blocks_per_stack = trial.suggest_int("blocks_per_stack", 1, 2),
        dropout          = trial.suggest_float("dropout", 0.05, 0.35),
        dense            = trial.suggest_categorical("dense", [32, 64, 128]),
        act              = trial.suggest_categorical("act", ["relu", "elu", "selu", tf.nn.swish]),
        lr               = trial.suggest_float("lr", 5e-5, 3e-3, log=True),
        batch            = trial.suggest_categorical("batch", [32, 64]),
        norm             = trial.suggest_categorical("norm", ["none", "batch", "layer"]),
    )

    X_tr, y_tr = make_windows(X_tr_raw, y_tr_raw, cfg["window"])
    X_va, y_va = make_windows(X_va_raw, y_va_raw, cfg["window"])

    if len(X_tr) < cfg["batch"] * 10:      # too small → prune
        return float("inf")

    tf.keras.backend.clear_session(); gc.collect()
    model = build_tcn(cfg)

    cb = [
        keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=0),
        keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, min_lr=1e-6, verbose=0),
        TFKerasPruningCallback(trial, "val_loss")
    ]

    hist = model.fit(X_tr, y_tr,
                     validation_data=(X_va, y_va),
                     epochs=100,
                     batch_size=cfg["batch"],
                     shuffle=False,
                     callbacks=cb,
                     verbose=0)

    y_prob = model.predict(X_va, batch_size=cfg["batch"], verbose=0).ravel()
    score  = f1_alpha2(y_va, y_prob)

    # store metrics to inspect later
    trial.set_user_attr("precision", precision_score(y_va, y_prob >= 0.5, zero_division=0))
    trial.set_user_attr("recall",    recall_score   (y_va, y_prob >= 0.5, zero_division=0))
    trial.set_user_attr("best_epoch", int(np.argmin(hist.history["val_loss"]) + 1))

    # print live metrics
    print(f"Trial {trial.number:2d}  "
          f"Fα2={score:.4f}  P={trial.user_attrs['precision']:.3f} "
          f"R={trial.user_attrs['recall']:.3f}  cfg={trial.params}")

    del model; tf.keras.backend.clear_session(); gc.collect()
    return -score   # minimise

# ─────────────────────────── main ──────────────────────────
def main():
    study = optuna.create_study(direction="minimize",
                                sampler=optuna.samplers.TPESampler(seed=SEED),
                                pruner =optuna.pruners.MedianPruner(n_startup_trials=5))

    print(f"\n🚀 Optimising  {N_TRIALS} trials  |  {TIMEOUT//60} min wall-time …")
    study.optimize(objective,
                   n_trials=N_TRIALS,
                   timeout=TIMEOUT,
                   show_progress_bar=True,
                   gc_after_trial=True)

    best, ts = study.best_trial, datetime.utcnow().strftime("%Y%m%d_%H%M%S")

    print("\n═══════════ BEST RESULT ═══════════")
    print(f"weighted-F1 (α=2) : {-best.value:.4f}")
    print(f"precision         : {best.user_attrs['precision']:.4f}")
    print(f"recall            : {best.user_attrs['recall']:.4f}")
    print(f"best epoch        : {best.user_attrs['best_epoch']}")
    print("hyper-parameters  :")
    for k, v in best.params.items():
        print(f"  {k:18s}: {v}")

    json.dump(best.params, open(f"best_params_tcn_{ts}.json", "w"), indent=2)
    study.trials_dataframe().to_csv(f"trials_tcn_{ts}.csv", index=False)

    try:
        import matplotlib.pyplot as plt
        optuna.visualization.matplotlib.plot_optimization_history(study)
        plt.tight_layout()
        plt.savefig(f"optuna_tcn_history_{ts}.png", dpi=300)
        plt.close()
    except Exception:
        print("⚠️ matplotlib unavailable – history plot skipped.")

    print(f"\n📝 Artefacts saved (timestamp {ts}).  "
          f"Scaler → {SCALER_PKL}")

# ───────────────────────────────────────────────────────────
if __name__ == "__main__":
    main()


📊 Loading data …
Shape  : (15855, 59)
Classes:
 target
1    8097
0    7758
Name: count, dtype: int64 

Class-weights 0 / 1 → 1.00 / 0.97


[I 2025-06-09 18:52:10,220] A new study created in memory with name: no-name-5d78d030-0093-439e-a2ba-35523e4ee0ab



🚀 Optimising  40 trials  |  80 min wall-time …


  0%|          | 0/40 [00:00<?, ?it/s]

Trial  0  Fα2=0.1605  P=0.509 R=0.120  cfg={'window': 30, 'filters': 32, 'kernel': 2, 'nb_stacks': 1, 'blocks_per_stack': 2, 'dropout': 0.23033450352296264, 'dense': 128, 'act': 'relu', 'lr': 0.00017376356936978755, 'batch': 32, 'norm': 'batch'}


...

Trial 36  Fα2=0.7666  P=0.523 R=1.000  cfg={'window': 12, 'filters': 48, 'kernel': 5, 'nb_stacks': 3, 'blocks_per_stack': 2, 'dropout': 0.3118914406542517, 'dense': 128, 'act': 'relu', 'lr': 0.0005947328436650964, 'batch': 32, 'norm': 'layer'}


Best trial: 4. Best value: -0.765667:  90%|█████████ | 36/40 [1:21:31<04:43, 70.99s/it, 3711.39/4800 seconds]

[I 2025-06-09 20:13:41,748] Trial 36 finished with value: -0.766599597585513 and parameters: {'window': 12, 'filters': 48, 'kernel': 5, 'nb_stacks': 3, 'blocks_per_stack': 2, 'dropout': 0.3118914406542517, 'dense': 128, 'act': 'relu', 'lr': 0.0005947328436650964, 'batch': 32, 'norm': 'layer'}. Best is trial 36 with value: -0.766599597585513.


Best trial: 36. Best value: -0.7666:  92%|█████████▎| 37/40 [1:21:34<06:36, 132.27s/it, 4894.06/4800 seconds]



═══════════ BEST RESULT ═══════════
weighted-F1 (α=2) : 0.7666
precision         : 0.5226
recall            : 1.0000
best epoch        : 35
hyper-parameters  :
  window            : 12
  filters           : 48
  kernel            : 5
  nb_stacks         : 3
  blocks_per_stack  : 2
  dropout           : 0.3118914406542517
  dense             : 128
  act               : relu
  lr                : 0.0005947328436650964
  batch             : 32
  norm              : layer

📝 Artefacts saved (timestamp 20250609_171344).  Scaler → tcn_scaler.pkl


In [6]:
import os, json, joblib, warnings
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             roc_auc_score, confusion_matrix, classification_report)

# ═══════════════ Setup ══════════════════════════════════════════════
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
for g in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

# ═══════════════ Configuration ══════════════════════════════════════
CSV_PATH = Path(r"C:/Users/ADMIN/Desktop/Coding_projects/stock_market_prediction/"
                r"Stock-Market-Prediction/data/processed/"
                r"gemini_btc_with_features_4h.csv")

# Output files
MODEL_OUT = "cnn_optimal_final.h5"
SCALER_OUT = "cnn_scaler_final.pkl"
PREDICTIONS_OUT = "cnn_predictions.csv"
SUMMARY_JSON = "cnn_final_training_summary.json"

# Optimal hyperparameters
OPTIMAL_PARAMS =    {
        'name': 'Trial_2_HighestPrecision',
        'window': 24, 'filters': 32, 'kernel': 2,
        'nb_stacks': 3, 'blocks_per_stack': 2,
        'dropout': 0.3318, 'dense': 128, 'act': 'swish',
        'lr': 0.0002455, 'batch': 64, 'norm': 'layer'
    }

VAL_FRAC = 0.20
EARLY_STOP = 15
ALPHA = 2.0

DROP_COLS = [
    'open', 'high', 'low', 'typical_price', 'EMA_21', 'SMA_20',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower',
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'trending_market', 'above_sma50', 'ema7_above_ema21',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold',
    'above_sma20', 'macd_positive', 'volume_breakout', 'volume_breakdown',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6','ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal',
]

# ═══════════════ Helper Functions ═══════════════════════════════════
def weighted_f1(y_true, y_pred, alpha=ALPHA):
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    return 0.0 if p + r == 0 else (1 + alpha) * p * r / (alpha * p + r)

def make_windows(arr, labels, win):
    xs, ys = [], []
    for i in range(win, len(arr)):
        xs.append(arr[i-win:i])
        ys.append(labels[i])
    return np.asarray(xs, np.float32), np.asarray(ys, np.int8)

def build_model(win, n_features):
    params = OPTIMAL_PARAMS
    inp = layers.Input(shape=(win, n_features))
    x = inp

    for b in range(params['conv_blocks']):
        f = params['filters'] * (2 ** b)
        y = layers.Conv1D(f, params['kernel'], padding="causal",
                          activation=params['act'],
                          kernel_regularizer=regularizers.l2(params['l2']))(x)
        y = layers.BatchNormalization()(y)
        if params['conv_blocks'] > 1:
            if x.shape[-1] != y.shape[-1]:
                x = layers.Conv1D(f, 1, padding="same")(x)
            x = layers.add([x, y])
        else:
            x = y

    if params['pool'] == 'gmp':
        x = layers.GlobalMaxPooling1D()(x)
    else:
        x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(params['dropout'])(x)

    if params['extra_dense']:
        x = layers.Dense(params['dense'], activation=params['act'],
                         kernel_regularizer=regularizers.l2(params['l2']))(x)
        x = layers.Dropout(params['dropout'])(x)

    out = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inp, out)

# ═══════════════ 1. Data Loading ═══════════════════════════════════
print("📊 Loading & preprocessing data...")
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
print("Before dropna:", df.shape)
df = df.loc["2018-01-01":]
df = df.drop(columns=[c for c in DROP_COLS if c in df.columns])
df = df.dropna(subset=["target"]).dropna()
print("After dropna:", df.shape)

X_raw = df.drop(columns="target").values
y_raw = df["target"].astype(int).values
n_features = X_raw.shape[1]

split_idx = int(len(df) * (1 - VAL_FRAC))
scaler = StandardScaler().fit(X_raw[:split_idx])

X_train_s = scaler.transform(X_raw[:split_idx]).astype(np.float32)
X_val_s = scaler.transform(X_raw[split_idx:]).astype(np.float32)
y_train = y_raw[:split_idx]
y_val = y_raw[split_idx:]

win = OPTIMAL_PARAMS['window']
X_train, y_train = make_windows(X_train_s, y_train, win)
X_val, y_val = make_windows(X_val_s, y_val, win)

# ═══════════════ 2. Model Building ══════════════════════════════════
model = build_model(win, n_features)
model.compile(optimizer=keras.optimizers.Adam(OPTIMAL_PARAMS['lr']),
              loss="binary_crossentropy",
              metrics=["accuracy"])

# ═══════════════ 3. Training ════════════════════════════════════════
callbacks = [
    keras.callbacks.EarlyStopping(patience=EARLY_STOP,
                                  restore_best_weights=True,
                                  verbose=1),
    keras.callbacks.ReduceLROnPlateau(factor=0.5,
                                      patience=5,
                                      verbose=1,
                                      min_lr=1e-7)
]
history = model.fit(X_train, y_train,
                    epochs=OPTIMAL_PARAMS['epochs'],
                    batch_size=OPTIMAL_PARAMS['batch'],
                    validation_data=(X_val, y_val),
                    callbacks=callbacks,
                    verbose=2)

# ═══════════════ 4. Evaluation ══════════════════════════════════════
prob_up = model.predict(X_val, verbose=0).ravel()
prob_down = 1 - prob_up
pred = (prob_up >= 0.5).astype(int)

precision = precision_score(y_val, pred, zero_division=0)
recall = recall_score(y_val, pred, zero_division=0)
wf1 = weighted_f1(y_val, pred)
accuracy = accuracy_score(y_val, pred)
auc = roc_auc_score(y_val, prob_up)

print(f"\n🎯 Trial Metrics — Precision: {precision:.4f}, Recall: {recall:.4f}, F1(α={ALPHA}): {wf1:.4f}")

# Remaining steps unchanged: saving predictions, model, summary JSON...


📊 Loading & preprocessing data...
Before dropna: (20718, 66)
After dropna: (15855, 34)


KeyError: 'conv_blocks'

In [ ]:
"""
tcn_precision_comparison.py
──────────────────────────────────────────────────
Trains and compares the top 3 TCN configurations that achieved
the highest precision scores during optimization.
"""

import os, json, gc, warnings
from datetime import datetime
from pathlib import Path
from typing import Dict, Tuple

import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (precision_score, recall_score, f1_score,
                             confusion_matrix, accuracy_score, roc_auc_score)

import tensorflow as tf
from tensorflow import keras
from tcn import TCN

# ─────────────────── Setup ───────────────────────────
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
for g in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

# ─────────────────── Configuration ───────────────────
CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")

VAL_FRAC = 0.20
ALPHA = 2.0
EPOCHS = 100
EARLY_STOP = 15

DROP_COLS = [
    'open', 'high', 'low', 'typical_price', 'EMA_21', 'SMA_20',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower',
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'trending_market', 'above_sma50', 'ema7_above_ema21',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold',
    'above_sma20', 'macd_positive', 'volume_breakout', 'volume_breakdown',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6', 'ema_cross_up', 'macd_cross_up', 
    'oversold_reversal', 'overbought_reversal'
]

PRECISION_CONFIGS = [
    {
        'name': 'Trial_2_HighestPrecision',
        'window': 24, 'filters': 32, 'kernel': 2, 'nb_stacks': 3, 'blocks_per_stack': 2,
        'dropout': 0.3318, 'dense': 128, 'act': 'swish', 
        'lr': 0.0002455, 'batch': 64, 'norm': 'layer',
        'expected_precision': 0.532, 'expected_recall': 0.456, 'expected_f1': 0.478
    }

]

def weighted_f1(y_true, y_pred, alpha=ALPHA):
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    return 0.0 if p + r == 0 else (1 + alpha) * p * r / (alpha * p + r)

def make_windows(arr, labels, win):
    X, y = [], []
    for i in range(win, len(arr)):
        X.append(arr[i-win:i])
        y.append(labels[i])
    return np.asarray(X, np.float32), np.asarray(y, np.int8)

def build_tcn_model(cfg, n_features, class_w0, class_w1):
    act_map = {'relu': 'relu', 'elu': 'elu', 'selu': 'selu', 'swish': tf.nn.swish, 'tanh': 'tanh'}
    act = act_map.get(cfg['act'], cfg['act'])
    dilations = [2 ** i for i in range(cfg['nb_stacks'] * cfg['blocks_per_stack'])]

    inputs = keras.layers.Input(shape=(cfg['window'], n_features))
    x = TCN(nb_filters=cfg['filters'], kernel_size=cfg['kernel'], nb_stacks=cfg['nb_stacks'],
            dilations=dilations, padding="causal", dropout_rate=cfg['dropout'],
            activation=act, use_skip_connections=True, use_batch_norm=cfg['norm'] == 'batch',
            use_layer_norm=cfg['norm'] == 'layer', return_sequences=False)(inputs)
    x = keras.layers.Dense(cfg['dense'], activation=act)(x)
    x = keras.layers.Dropout(cfg['dropout'])(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs, outputs)

    def weighted_bce(y_t, y_p):
        y_t = tf.cast(y_t, y_p.dtype)
        w = tf.where(tf.equal(y_t, 1), class_w1, class_w0)
        w = tf.cast(w, y_p.dtype)
        return tf.reduce_mean(w * keras.losses.binary_crossentropy(y_t, y_p))

    model.compile(optimizer=keras.optimizers.Adam(cfg['lr']), loss=weighted_bce)
    return model

# ─────────────────── Load Data ───────────────────────
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df.loc["2018-01-01":]
df = df.drop(columns=[c for c in DROP_COLS if c in df.columns])
df = df.dropna(subset=["target"]).dropna()

X_raw = df.drop(columns="target").values
y_raw = df["target"].astype(int).values
n_features = X_raw.shape[1]
split_idx = int(len(df) * (1 - VAL_FRAC))
scaler = StandardScaler().fit(X_raw[:split_idx])

X_train = scaler.transform(X_raw[:split_idx]).astype(np.float32)
X_val = scaler.transform(X_raw[split_idx:]).astype(np.float32)
y_train, y_val = y_raw[:split_idx], y_raw[split_idx:]

pos_rate = y_train.mean()
CLASS_W0 = np.float32(1.0)
CLASS_W1 = np.float32((1 - pos_rate) / pos_rate if pos_rate else 1.0)

# ─────────────────── Training Loop ───────────────────
results = []

for cfg in PRECISION_CONFIGS:
    print(f"\n🔬 {cfg['name']} | Window={cfg['window']}, Filters={cfg['filters']} | act={cfg['act']}, norm={cfg['norm']}")

    X_tr, y_tr = make_windows(X_train, y_train, cfg['window'])
    X_va, y_va = make_windows(X_val, y_val, cfg['window'])

    tf.keras.backend.clear_session(); gc.collect()

    model = build_tcn_model(cfg, n_features, CLASS_W0, CLASS_W1)

    callbacks = [
        keras.callbacks.EarlyStopping(patience=EARLY_STOP, restore_best_weights=True, verbose=0),
        keras.callbacks.ReduceLROnPlateau(patience=7, factor=0.5, min_lr=1e-7, verbose=0)
    ]

    model.fit(X_tr, y_tr, validation_data=(X_va, y_va), epochs=EPOCHS,
              batch_size=cfg['batch'], shuffle=False, callbacks=callbacks, verbose=0)

    prob = model.predict(X_va, verbose=0).ravel()
    pred = (prob >= 0.5).astype(int)

    precision = precision_score(y_va, pred, zero_division=0)
    recall = recall_score(y_va, pred, zero_division=0)
    f1_std = f1_score(y_va, pred, zero_division=0)
    f1_wgt = weighted_f1(y_va, pred)
    acc = accuracy_score(y_va, pred)
    auc = roc_auc_score(y_va, prob)

    print(f"   Precision: {precision:.3f} | Recall: {recall:.3f} | F1: {f1_std:.3f} | F1α=2: {f1_wgt:.3f} | AUC: {auc:.3f}")

    results.append({
        "name": cfg["name"],
        "actual_precision": precision,
        "actual_recall": recall,
        "actual_f1": f1_std,
        "actual_f1_weighted": f1_wgt,
        "actual_accuracy": acc,
        "actual_auc": auc
    })

# ─────────────────── Summary ─────────────────────────
print("\n🏁 FINAL RANKING BY PRECISION:")
for i, res in enumerate(sorted(results, key=lambda r: r['actual_precision'], reverse=True), 1):
    print(f"{i}. {res['name']:<25s} → P={res['actual_precision']:.3f}, R={res['actual_recall']:.3f}, F1={res['actual_f1_weighted']:.3f}, AUC={res['actual_auc']:.3f}")



🔬 Trial_2_HighestPrecision | Window=24, Filters=32 | act=swish, norm=layer
   Precision: 0.531 | Recall: 0.709 | F1: 0.607 | F1α=2: 0.638 | AUC: 0.510

🔬 Trial_3_SecondPrecision | Window=18, Filters=64 | act=selu, norm=batch
   Precision: 0.541 | Recall: 0.147 | F1: 0.231 | F1α=2: 0.194 | AUC: 0.512

🔬 Trial_36_BestOverall | Window=12, Filters=48 | act=relu, norm=layer
   Precision: 0.523 | Recall: 1.000 | F1: 0.686 | F1α=2: 0.767 | AUC: 0.511

🏁 FINAL RANKING BY PRECISION:
1. Trial_3_SecondPrecision   → P=0.541, R=0.147, F1=0.194, AUC=0.512
2. Trial_2_HighestPrecision  → P=0.531, R=0.709, F1=0.638, AUC=0.510
3. Trial_36_BestOverall      → P=0.523, R=1.000, F1=0.767, AUC=0.511


In [ ]:
 [
    'open', 'high', 'low', 'typical_price', 'EMA_21', 'SMA_20',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower',
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'trending_market', 'above_sma50', 'ema7_above_ema21',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold',
    'above_sma20', 'macd_positive', 'volume_breakout', 'volume_breakdown',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6','ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal',
]

In [7]:
"""
tcn_precision_comparison.py
────────────────────────────────────────────────────────────
Compares hand-picked TCN configs on BTC-4 h data and
saves *per-bar* validation predictions in the format:

timestamp,prob_up,prob_down,winning_prob,prediction,actual
20/10/2023 16:00,0.520832,0.479168,0.520832,1,1
…
"""
# ───────────────────────── imports ─────────────────────────
import os, json, gc, warnings
from datetime import datetime
from pathlib import Path

import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score, roc_auc_score)
import tensorflow as tf
from tensorflow import keras
from tcn import TCN

# ───────────────────── runtime hygiene ─────────────────────
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
for g in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

# ───────────────────────── config ──────────────────────────
CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")

VAL_FRAC   = 0.20          # last 20 % is hold-out
ALPHA      = 2.0           # precision weight for weighted-F1
EPOCHS     = 100
EARLY_STOP = 15

DROP_COLS =  [
    'open', 'high', 'low', 'typical_price', 'EMA_21', 'SMA_20',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower',
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'trending_market', 'above_sma50', 'ema7_above_ema21',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold',
    'above_sma20', 'macd_positive', 'volume_breakout', 'volume_breakdown',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6','ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal',
]          # ← paste the long list you already had

PRECISION_CONFIGS = [
    {
        'name': 'Trial_2_HighestPrecision',
        'window': 24, 'filters': 32, 'kernel': 2,
        'nb_stacks': 3, 'blocks_per_stack': 2,
        'dropout': 0.3318, 'dense': 128, 'act': 'swish',
        'lr': 0.0002455, 'batch': 64, 'norm': 'layer'
    },
    # add more configs if you like …
]

# ───────────────────────── helpers ─────────────────────────
def weighted_f1(y_t, y_p, a=ALPHA):
    p = precision_score(y_t, y_p, zero_division=0)
    r = recall_score   (y_t, y_p, zero_division=0)
    return 0.0 if p+r == 0 else (1+a)*p*r/(a*p+r)

def make_windows(x, y, win):
    xs, ys = [], []
    for i in range(win, len(x)):
        xs.append(x[i-win:i])
        ys.append(y[i])
    return np.asarray(xs, np.float32), np.asarray(ys, np.int8)

def build_tcn(cfg, n_feat, cw0, cw1):
    act_map = {'relu':'relu','elu':'elu','selu':'selu','swish':tf.nn.swish,
               'tanh':'tanh'}
    act = act_map[cfg['act']]
    dilations = [2**i for i in range(cfg['nb_stacks']*cfg['blocks_per_stack'])]

    inp = keras.Input((cfg['window'], n_feat))
    x   = TCN(nb_filters=cfg['filters'],
              kernel_size=cfg['kernel'],
              nb_stacks=cfg['nb_stacks'],
              dilations=dilations,
              padding='causal',
              dropout_rate=cfg['dropout'],
              activation=act,
              use_skip_connections=True,
              use_batch_norm = cfg['norm']=='batch',
              use_layer_norm = cfg['norm']=='layer',
              return_sequences=False)(inp)

    x   = keras.layers.Dense(cfg['dense'], activation=act)(x)
    x   = keras.layers.Dropout(cfg['dropout'])(x)
    out = keras.layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inp, out)

    # class-balanced BCE
    def weighted_bce(y_t, y_p):
        y_t = tf.cast(y_t, y_p.dtype)
        w   = tf.where(tf.equal(y_t, 1), cw1, cw0)
        return tf.reduce_mean(w * keras.losses.binary_crossentropy(y_t, y_p))

    model.compile(keras.optimizers.Adam(cfg['lr']), loss=weighted_bce)
    return model

# ─────────────────────── load data once ───────────────────
print("📊 loading & scaling data …")
df = (pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
        .loc["2018-01-01":]
        .drop(columns=[c for c in DROP_COLS if c in df.columns])
        .dropna(subset=['target']).dropna())

X, y = df.drop(columns='target').values, df['target'].astype(int).values
n_features = X.shape[1]
split = int(len(df)*(1-VAL_FRAC))

scaler = StandardScaler().fit(X[:split])
X_tr_raw = scaler.transform(X[:split]).astype(np.float32)
X_va_raw = scaler.transform(X[split:]).astype(np.float32)
y_tr_raw, y_va_raw = y[:split], y[split:]

pos_rate = y_tr_raw.mean()
CW0 = np.float32(1.0)
CW1 = np.float32((1-pos_rate)/pos_rate if pos_rate else 1.0)

# ───────────────────── compare configs ─────────────────────
results = []

for cfg in PRECISION_CONFIGS:
    tag = (f"{cfg['name']} | win={cfg['window']}, "
           f"filters={cfg['filters']}, act={cfg['act']}")
    print("\n"+tag+"\n"+"─"*len(tag))

    # build windows
    X_tr, y_tr = make_windows(X_tr_raw, y_tr_raw, cfg['window'])

    # leak-free: prepend last ‘window’ rows of train to val before windowing
    X_val_stack = np.concatenate([X_tr_raw[-cfg['window']:], X_va_raw])
    y_val_stack = np.concatenate([y_tr_raw[-cfg['window']:], y_va_raw])
    X_va, y_va = make_windows(X_val_stack, y_val_stack, cfg['window'])

    tf.keras.backend.clear_session(); gc.collect()
    model = build_tcn(cfg, n_features, CW0, CW1)

    callbacks = [
        keras.callbacks.EarlyStopping(patience=EARLY_STOP,
                                      restore_best_weights=True, verbose=0),
        keras.callbacks.ReduceLROnPlateau(patience=7, factor=.5,
                                          min_lr=1e-7, verbose=0)
    ]

    model.fit(X_tr, y_tr,
              validation_data=(X_va, y_va),
              epochs=EPOCHS,
              batch_size=cfg['batch'],
              shuffle=False,
              callbacks=callbacks,
              verbose=0)

    prob = model.predict(X_va, verbose=0).ravel()
    pred = (prob >= 0.5).astype(int)

    prec = precision_score(y_va, pred, zero_division=0)
    rec  = recall_score   (y_va, pred, zero_division=0)
    f1   = f1_score       (y_va, pred, zero_division=0)
    f1_w = weighted_f1    (y_va, pred)
    auc  = roc_auc_score  (y_va, prob)
    acc  = accuracy_score (y_va, pred)

    print(f"P={prec:.3f}  R={rec:.3f}  F1={f1:.3f} "
          f"Fα2={f1_w:.3f}  AUC={auc:.3f}")

    # ─── save per-bar predictions in required format ──
    val_start = split + cfg['window']
    ts_index  = df.index[val_start : val_start + len(prob)]

    # ─── save per-bar predictions in required format ──
    val_start = split                          # ← FIXED
    ts_index  = df.index[val_start : val_start + len(prob)]

    pred_df = pd.DataFrame({
        "timestamp"   : ts_index.strftime("%d/%m/%Y %H:%M"),
        "prob_up"     : prob,
        "prob_down"   : 1.0 - prob,
        "winning_prob": np.where(prob >= 0.5, prob, 1.0 - prob),
        "prediction"  : pred,
        "actual"      : y_va
    })

    csv_name = f"{cfg['name']}_win{cfg['window']}_predictions.csv"
    pred_df.to_csv(csv_name, index=False, float_format="%.6f")
    print(f"📄 saved predictions → {csv_name}")


    csv_name = f"{cfg['name']}_win{cfg['window']}_predictions.csv"
    pred_df.to_csv(csv_name, index=False, float_format="%.6f")
    print(f"📄 saved predictions → {csv_name}")

    # keep run-level metrics
    results.append(dict(name=cfg['name'], precision=prec,
                        recall=rec, f1=f1_w, auc=auc))

# ───────────────────── leaderboard ────────────────────────
print("\n🏆 Leaderboard (by Precision)")
for rk, res in enumerate(sorted(results, key=lambda x: x['precision'],
                                reverse=True), 1):
    print(f"{rk}. {res['name']:<25s} | "
          f"P={res['precision']:.3f} R={res['recall']:.3f} "
          f"Fα2={res['f1']:.3f} AUC={res['auc']:.3f}")

# optional: JSON summary
with open("tcn_comparison_summary.json", "w") as fp:
    json.dump({"timestamp":datetime.utcnow().isoformat(timespec='seconds')+'Z',
               "metrics":results}, fp, indent=2)
print("\n📑 Summary saved → tcn_comparison_summary.json")


📊 loading & scaling data …

Trial_2_HighestPrecision | win=24, filters=32, act=swish
────────────────────────────────────────────────────────


KeyboardInterrupt: 

In [ ]:
"""
tcn_trial3_only.py - TRIAL 3 SECOND PRECISION
──────────────────────────────────────────────
Quick training of Trial 3 configuration only.
Expected: Precision=0.527, Recall=0.300, F1=0.350
"""

import os, json, gc, warnings
from datetime import datetime
from pathlib import Path

import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score, roc_auc_score)
import tensorflow as tf
from tensorflow import keras
from tcn import TCN

# ───────────────────── Setup ───────────────────────────────
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
for g in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

print("🖥️ GPU Status:")
gpus = tf.config.list_physical_devices('GPU')
print(f"   GPUs available: {len(gpus)}")

# ───────────────────── Config ──────────────────────────────
CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")

VAL_FRAC = 0.20
ALPHA = 2.0
EPOCHS = 60  # Quick training
EARLY_STOP = 20

DROP_COLS = [
    'open', 'high', 'low', 'typical_price', 'EMA_21', 'SMA_20',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower',
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'trending_market', 'above_sma50', 'ema7_above_ema21',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold',
    'above_sma20', 'macd_positive', 'volume_breakout', 'volume_breakdown',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6', 'ema_cross_up', 'macd_cross_up', 
    'oversold_reversal', 'overbought_reversal'
]

# TRIAL 3 CONFIG ONLY
CFG =    {
        'name': 'Trial_2_HighestPrecision',
        'window': 24, 'filters': 32, 'kernel': 2,
        'nb_stacks': 3, 'blocks_per_stack': 2,
        'dropout': 0.3318, 'dense': 128, 'act': 'swish',
        'lr': 0.0002455, 'batch': 64, 'norm': 'layer'
    }

print(f"🎯 Training: {CFG['name']}")
print(f"   Expected: P=0.527, R=0.300, F1=0.350")
print(f"   Window: {CFG['window']}, Filters: {CFG['filters']}, Act: {CFG['act']}")

# ───────────────────── Helpers ─────────────────────────────
def weighted_f1(y_t, y_p, a=ALPHA):
    p = precision_score(y_t, y_p, zero_division=0)
    r = recall_score(y_t, y_p, zero_division=0)
    return 0.0 if p+r == 0 else (1+a)*p*r/(a*p+r)

def make_windows(x, y, win):
    xs, ys = [], []
    for i in range(win, len(x)):
        xs.append(x[i-win:i])
        ys.append(y[i])
    return np.asarray(xs, np.float32), np.asarray(ys, np.int8)

def build_tcn(cfg, n_feat, cw0, cw1):
    act_map = {'relu':'relu', 'elu':'elu', 'selu':'selu', 'swish':tf.nn.swish, 'tanh':'tanh'}
    act = act_map[cfg['act']]
    dilations = [2**i for i in range(cfg['nb_stacks'] * cfg['blocks_per_stack'])]

    inp = keras.Input((cfg['window'], n_feat))
    x = TCN(nb_filters=cfg['filters'],
            kernel_size=cfg['kernel'],
            nb_stacks=cfg['nb_stacks'],
            dilations=dilations,
            padding='causal',
            dropout_rate=cfg['dropout'],
            activation=act,
            use_skip_connections=True,
            use_batch_norm=cfg['norm']=='batch',
            use_layer_norm=cfg['norm']=='layer',
            return_sequences=False)(inp)

    x = keras.layers.Dense(cfg['dense'], activation=act)(x)
    x = keras.layers.Dropout(cfg['dropout'])(x)
    out = keras.layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inp, out)

    # Weighted BCE
    def weighted_bce(y_t, y_p):
        y_t = tf.cast(y_t, y_p.dtype)
        w = tf.where(tf.equal(y_t, 1), cw1, cw0)
        return tf.reduce_mean(w * keras.losses.binary_crossentropy(y_t, y_p))

    model.compile(keras.optimizers.Adam(cfg['lr']), loss=weighted_bce)
    return model

# ───────────────────── Load Data ──────────────────────────
print("\n📊 Loading data...")
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df.loc["2018-01-01":]
df = df.drop(columns=[c for c in DROP_COLS if c in df.columns])
df = df.dropna(subset=["target"]).dropna()


X, y = df.drop(columns='target').values, df['target'].astype(int).values
n_features = X.shape[1]
split = int(len(df) * (1 - VAL_FRAC))

print(f"   Data shape: {df.shape}")
print(f"   Features: {n_features}")
print(f"   Train/Val split: {split}/{len(df)-split}")

# Scale
scaler = StandardScaler().fit(X[:split])
X_tr_raw = scaler.transform(X[:split]).astype(np.float32)
X_va_raw = scaler.transform(X[split:]).astype(np.float32)
y_tr_raw, y_va_raw = y[:split], y[split:]

# Class weights
pos_rate = y_tr_raw.mean()
CW0 = np.float32(1.0)
CW1 = np.float32((1-pos_rate)/pos_rate if pos_rate else 1.0)
print(f"   Class weights: {CW0:.2f} / {CW1:.2f}")

# ───────────────────── Train Model ────────────────────────
print(f"\n🚀 Training {CFG['name']}...")

# Create windows
X_tr, y_tr = make_windows(X_tr_raw, y_tr_raw, CFG['window'])
X_va, y_va = make_windows(X_va_raw, y_va_raw, CFG['window'])

print(f"   Train windows: {len(X_tr):,}")
print(f"   Val windows: {len(X_va):,}")

# Build model
tf.keras.backend.clear_session()
gc.collect()
model = build_tcn(CFG, n_features, CW0, CW1)

# Callbacks
callbacks = [
    keras.callbacks.EarlyStopping(patience=EARLY_STOP, restore_best_weights=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.5, min_lr=1e-7, verbose=1)
]

# Train
start_time = datetime.now()
history = model.fit(X_tr, y_tr,
                    validation_data=(X_va, y_va),
                    epochs=EPOCHS,
                    batch_size=CFG['batch'],
                    shuffle=False,
                    callbacks=callbacks,
                    verbose=2)

train_time = datetime.now() - start_time
print(f"\n⏱️ Training completed in: {train_time}")

# ───────────────────── Evaluate ───────────────────────────
print("\n📊 Evaluating...")
prob = model.predict(X_va, verbose=0).ravel()
pred = (prob >= 0.5).astype(int)

prec = precision_score(y_va, pred, zero_division=0)
rec = recall_score(y_va, pred, zero_division=0)
f1 = f1_score(y_va, pred, zero_division=0)
f1_w = weighted_f1(y_va, pred)
auc = roc_auc_score(y_va, prob)
acc = accuracy_score(y_va, pred)

print(f"\n📈 Results vs Expected:")
print(f"   Precision: {prec:.3f} (expected: 0.527) {'✅' if prec >= 0.520 else '❌'}")
print(f"   Recall:    {rec:.3f} (expected: 0.300) {'✅' if rec >= 0.290 else '❌'}")
print(f"   F1 Std:    {f1:.3f} (expected: 0.350)")
print(f"   F1 Weighted (α=2): {f1_w:.3f}")
print(f"   Accuracy:  {acc:.3f}")
print(f"   AUC:       {auc:.3f}")

# ───────────────────── Save Results ───────────────────────
print("\n💾 Saving results...")

# Save model and scaler
model.save(f"tcn_{CFG['name'].lower()}.h5")
import joblib
joblib.dump(scaler, f"scaler_{CFG['name'].lower()}.pkl")

# Generate predictions CSV
val_start_idx = split + CFG['window']
timestamps = df.index[val_start_idx:val_start_idx + len(prob)]

pred_df = pd.DataFrame({
    "timestamp": timestamps.strftime("%d/%m/%Y %H:%M"),
    "prob_up": prob,
    "prob_down": 1.0 - prob,
    "winning_prob": np.maximum(prob, 1.0 - prob),
    "prediction": pred,
    "actual": y_va
})

csv_name = f"{CFG['name']}_predictions.csv"
pred_df.to_csv(csv_name, index=False, float_format="%.6f")

print(f"✅ Model saved: tcn_{CFG['name'].lower()}.h5")
print(f"✅ Scaler saved: scaler_{CFG['name'].lower()}.pkl") 
print(f"✅ Predictions saved: {csv_name} ({len(pred_df):,} rows)")

# Save summary
summary = {
    "timestamp": datetime.utcnow().isoformat() + "Z",
    "config": CFG,
    "training_time_seconds": train_time.total_seconds(),
    "epochs_trained": len(history.history['loss']),
    "metrics": {
        "precision": float(prec),
        "recall": float(rec),
        "f1_standard": float(f1),
        "f1_weighted": float(f1_w),
        "accuracy": float(acc),
        "auc": float(auc)
    },
    "expected_vs_actual": {
        "precision_diff": float(prec - 0.527),
        "recall_diff": float(rec - 0.300),
        "f1_diff": float(f1 - 0.350)
    }
}

with open(f"{CFG['name']}_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print(f"✅ Summary saved: {CFG['name']}_summary.json")
print(f"\n🎉 Training complete! Time: {train_time}")

# Quick sample of predictions
print(f"\n📋 Sample predictions:")
print(pred_df.head(5).to_string(index=False))

🖥️ GPU Status:
   GPUs available: 0
🎯 Training: Trial_2_HighestPrecision
   Expected: P=0.527, R=0.300, F1=0.350
   Window: 24, Filters: 32, Act: swish

📊 Loading data...
   Data shape: (15855, 34)
   Features: 33
   Train/Val split: 12684/3171
   Class weights: 1.00 / 0.97

🚀 Training Trial_2_HighestPrecision...
   Train windows: 12,660
   Val windows: 3,147

Epoch 1/60
198/198 - 31s - 155ms/step - loss: 0.9185 - val_loss: 0.6997 - learning_rate: 2.4550e-04
Epoch 2/60
198/198 - 17s - 87ms/step - loss: 0.7636 - val_loss: 0.6991 - learning_rate: 2.4550e-04
Epoch 3/60
198/198 - 17s - 86ms/step - loss: 0.7231 - val_loss: 0.6958 - learning_rate: 2.4550e-04
Epoch 4/60
198/198 - 17s - 86ms/step - loss: 0.7087 - val_loss: 0.6943 - learning_rate: 2.4550e-04
Epoch 5/60
198/198 - 17s - 86ms/step - loss: 0.6985 - val_loss: 0.6932 - learning_rate: 2.4550e-04
Epoch 6/60
198/198 - 17s - 86ms/step - loss: 0.6951 - val_loss: 0.6955 - learning_rate: 2.4550e-04
Epoch 7/60
198/198 - 17s - 86ms/step - los


📈 Results vs Expected:
   Precision: 0.534 (expected: 0.527) ✅
   Recall:    0.269 (expected: 0.300) ❌
   F1 Std:    0.358 (expected: 0.350)
   F1 Weighted (α=2): 0.322
   Accuracy:  0.495
   AUC:       0.512

💾 Saving results...


NotImplementedError: 
Object ResidualBlock was created by passing
non-serializable argument values in `__init__()`,
and therefore the object must override `get_config()` in
order to be serializable. Please implement `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2, **kwargs):
        super().__init__(**kwargs)
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config

In [3]:
"""
tcn_trial3_only.py - TRIAL 3 SECOND PRECISION
Quick training of Trial 3 configuration only.
Expected: Precision=0.527, Recall=0.300, F1=0.350
"""

import os, json, gc, warnings
from datetime import datetime
from pathlib import Path

import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score, roc_auc_score)
import tensorflow as tf
from tensorflow import keras
from tcn import TCN

# ─────────────── Setup ───────────────
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
for g in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

print("🖥️ GPU Status:")
gpus = tf.config.list_physical_devices('GPU')
print(f"   GPUs available: {len(gpus)}")

# ─────────────── Config ───────────────
CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")

VAL_FRAC = 0.20
ALPHA = 2.0
EPOCHS = 60
EARLY_STOP = 20

DROP_COLS = [
    'open', 'high', 'low', 'typical_price', 'EMA_21', 'SMA_20',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower',
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'trending_market', 'above_sma50', 'ema7_above_ema21',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold',
    'above_sma20', 'macd_positive', 'volume_breakout', 'volume_breakdown',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6', 'ema_cross_up', 'macd_cross_up', 
    'oversold_reversal', 'overbought_reversal'
]

CFG = {
    'name': 'Trial_2_HighestPrecision',
    'window': 24, 'filters': 32, 'kernel': 2,
    'nb_stacks': 3, 'blocks_per_stack': 2,
    'dropout': 0.3318, 'dense': 128, 'act': 'swish',
    'lr': 0.0002455, 'batch': 64, 'norm': 'layer'
}

# ─────────────── Helpers ───────────────
def weighted_f1(y_t, y_p, a=ALPHA):
    p = precision_score(y_t, y_p, zero_division=0)
    r = recall_score(y_t, y_p, zero_division=0)
    return 0.0 if p+r == 0 else (1+a)*p*r/(a*p+r)

def make_windows(x, y, win):
    xs, ys = [], []
    for i in range(win, len(x)):
        xs.append(x[i-win:i])
        ys.append(y[i])
    return np.asarray(xs, np.float32), np.asarray(ys, np.int8)

def build_tcn(cfg, n_feat, cw0, cw1):
    act_map = {'relu':'relu', 'elu':'elu', 'selu':'selu', 'swish':tf.nn.swish, 'tanh':'tanh'}
    act = act_map[cfg['act']]
    dilations = [2**i for i in range(cfg['nb_stacks'] * cfg['blocks_per_stack'])]

    inp = keras.Input((cfg['window'], n_feat))
    x = TCN(nb_filters=cfg['filters'], kernel_size=cfg['kernel'], nb_stacks=cfg['nb_stacks'],
            dilations=dilations, padding='causal', dropout_rate=cfg['dropout'], activation=act,
            use_skip_connections=True, use_batch_norm=cfg['norm']=='batch', use_layer_norm=cfg['norm']=='layer',
            return_sequences=False)(inp)
    x = keras.layers.Dense(cfg['dense'], activation=act)(x)
    x = keras.layers.Dropout(cfg['dropout'])(x)
    out = keras.layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inp, out)

    def weighted_bce(y_t, y_p):
        y_t = tf.cast(y_t, y_p.dtype)
        w = tf.where(tf.equal(y_t, 1), cw1, cw0)
        return tf.reduce_mean(w * keras.losses.binary_crossentropy(y_t, y_p))

    model.compile(keras.optimizers.Adam(cfg['lr']), loss=weighted_bce)
    return model

# ─────────────── Load Data ───────────────
print("\n📊 Loading data...")
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df.loc["2018-01-01":]
df = df.drop(columns=[c for c in DROP_COLS if c in df.columns])
df = df.dropna(subset=["target"]).dropna()

X, y = df.drop(columns='target').values, df['target'].astype(int).values
n_features = X.shape[1]
split = int(len(df) * (1 - VAL_FRAC))

scaler = StandardScaler().fit(X[:split])
X_tr_raw = scaler.transform(X[:split]).astype(np.float32)
X_va_raw = scaler.transform(X[split:]).astype(np.float32)
y_tr_raw, y_va_raw = y[:split], y[split:]

# Class weights
pos_rate = y_tr_raw.mean()
CW0 = np.float32(1.0)
CW1 = np.float32((1-pos_rate)/pos_rate if pos_rate else 1.0)

# ─────────────── Train ───────────────
X_tr, y_tr = make_windows(X_tr_raw, y_tr_raw, CFG['window'])
X_va, y_va = make_windows(X_va_raw, y_va_raw, CFG['window'])

model = build_tcn(CFG, n_features, CW0, CW1)

callbacks = [
    keras.callbacks.EarlyStopping(patience=EARLY_STOP, restore_best_weights=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.5, min_lr=1e-7, verbose=1)
]

start_time = datetime.now()
model.fit(X_tr, y_tr, validation_data=(X_va, y_va),
          epochs=EPOCHS, batch_size=CFG['batch'], shuffle=False,
          callbacks=callbacks, verbose=2)
train_time = datetime.now() - start_time

# ─────────────── Evaluate ───────────────
prob = model.predict(X_va, verbose=0).ravel()
pred = (prob >= 0.5).astype(int)

prec = precision_score(y_va, pred, zero_division=0)
rec = recall_score(y_va, pred, zero_division=0)
f1 = f1_score(y_va, pred, zero_division=0)
f1_w = weighted_f1(y_va, pred)
auc = roc_auc_score(y_va, prob)
acc = accuracy_score(y_va, pred)

print(f"\n📈 Results:")
print(f"   Precision:  {prec:.3f}")
print(f"   Recall:     {rec:.3f}")
print(f"   F1 Score:   {f1:.3f}")
print(f"   F1 (α=2):   {f1_w:.3f}")
print(f"   Accuracy:   {acc:.3f}")
print(f"   AUC:        {auc:.3f}")

# ─────────────── Save Predictions ───────────────
val_start_idx = split + CFG['window']
timestamps = df.index[val_start_idx:val_start_idx + len(prob)]

pred_df = pd.DataFrame({
    "timestamp": timestamps.strftime("%d/%m/%Y %H:%M"),
    "prob_up": prob,
    "prob_down": 1.0 - prob,
    "winning_prob": np.maximum(prob, 1.0 - prob),
    "prediction": pred,
    "actual": y_va
})

csv_name = f"{CFG['name']}_predictions.csv"
pred_df.to_csv(csv_name, index=False, float_format="%.6f")
print(f"✅ Predictions saved: {csv_name} ({len(pred_df):,} rows)")


🖥️ GPU Status:
   GPUs available: 0

📊 Loading data...
Epoch 1/60
198/198 - 26s - 130ms/step - loss: 0.9407 - val_loss: 0.6833 - learning_rate: 2.4550e-04
Epoch 2/60
198/198 - 9s - 46ms/step - loss: 0.7668 - val_loss: 0.6830 - learning_rate: 2.4550e-04
Epoch 3/60
198/198 - 9s - 46ms/step - loss: 0.7286 - val_loss: 0.6820 - learning_rate: 2.4550e-04
Epoch 4/60
198/198 - 9s - 45ms/step - loss: 0.7065 - val_loss: 0.6822 - learning_rate: 2.4550e-04
Epoch 5/60
198/198 - 9s - 45ms/step - loss: 0.6947 - val_loss: 0.6826 - learning_rate: 2.4550e-04
Epoch 6/60
198/198 - 9s - 46ms/step - loss: 0.6913 - val_loss: 0.6841 - learning_rate: 2.4550e-04
Epoch 7/60
198/198 - 9s - 46ms/step - loss: 0.6907 - val_loss: 0.6838 - learning_rate: 2.4550e-04
Epoch 8/60

Epoch 8: ReduceLROnPlateau reducing learning rate to 0.00012275000335648656.
198/198 - 12s - 62ms/step - loss: 0.6903 - val_loss: 0.6832 - learning_rate: 2.4550e-04
Epoch 9/60
198/198 - 17s - 86ms/step - loss: 0.6880 - val_loss: 0.6819 - learnin

In [4]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the predictions CSV
csv_path = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\src\Models\models\models\Trial_2_HighestPrecision_predictions.csv"
df = pd.read_csv(csv_path)

# Ensure column names are correct and lowercase
df.columns = df.columns.str.strip().str.lower()

# Extract actual and predicted values
y_true = df['actual']
y_pred = df['prediction']  # prediction at threshold 0.5

# Calculate metrics
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Print results
print("📊 Evaluation at threshold 0.5:")
print(f"Precision: {precision:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"F1 Score : {f1:.3f}")

📊 Evaluation at threshold 0.5:
Precision: 0.525
Recall   : 0.705
F1 Score : 0.602


In [5]:
"""
tcn_trial3_only.py - TRIAL 3 SECOND PRECISION
Quick training of Trial 3 configuration only.
Expected: Precision=0.527, Recall=0.300, F1=0.350
"""

import os, json, gc, warnings
from datetime import datetime
from pathlib import Path

import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score, roc_auc_score)
import tensorflow as tf
from tensorflow import keras
from tcn import TCN

# ─────────────── Setup ───────────────
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
for g in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

print("🖥️ GPU Status:")
gpus = tf.config.list_physical_devices('GPU')
print(f"   GPUs available: {len(gpus)}")

# ─────────────── Config ───────────────
CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")

VAL_FRAC = 0.20
ALPHA = 2.0
EPOCHS = 60
EARLY_STOP = 20

DROP_COLS = [
    'open', 'high', 'low', 'typical_price', 'EMA_21', 'SMA_20',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower',
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'trending_market', 'above_sma50', 'ema7_above_ema21',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold',
    'above_sma20', 'macd_positive', 'volume_breakout', 'volume_breakdown',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6', 'ema_cross_up', 'macd_cross_up', 
    'oversold_reversal', 'overbought_reversal'
]

CFG =     {
        'name': 'Trial_36_BestOverall',
        'window': 12, 'filters': 48, 'kernel': 5,
        'nb_stacks': 3, 'blocks_per_stack': 2,
        'dropout': 0.312, 'dense': 128, 'act': 'relu',
        'lr': 0.000595, 'batch': 32, 'norm': 'layer'
    }

# ─────────────── Helpers ───────────────
def weighted_f1(y_t, y_p, a=ALPHA):
    p = precision_score(y_t, y_p, zero_division=0)
    r = recall_score(y_t, y_p, zero_division=0)
    return 0.0 if p+r == 0 else (1+a)*p*r/(a*p+r)

def make_windows(x, y, win):
    xs, ys = [], []
    for i in range(win, len(x)):
        xs.append(x[i-win:i])
        ys.append(y[i])
    return np.asarray(xs, np.float32), np.asarray(ys, np.int8)

def build_tcn(cfg, n_feat, cw0, cw1):
    act_map = {'relu':'relu', 'elu':'elu', 'selu':'selu', 'swish':tf.nn.swish, 'tanh':'tanh'}
    act = act_map[cfg['act']]
    dilations = [2**i for i in range(cfg['nb_stacks'] * cfg['blocks_per_stack'])]

    inp = keras.Input((cfg['window'], n_feat))
    x = TCN(nb_filters=cfg['filters'], kernel_size=cfg['kernel'], nb_stacks=cfg['nb_stacks'],
            dilations=dilations, padding='causal', dropout_rate=cfg['dropout'], activation=act,
            use_skip_connections=True, use_batch_norm=cfg['norm']=='batch', use_layer_norm=cfg['norm']=='layer',
            return_sequences=False)(inp)
    x = keras.layers.Dense(cfg['dense'], activation=act)(x)
    x = keras.layers.Dropout(cfg['dropout'])(x)
    out = keras.layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inp, out)

    def weighted_bce(y_t, y_p):
        y_t = tf.cast(y_t, y_p.dtype)
        w = tf.where(tf.equal(y_t, 1), cw1, cw0)
        return tf.reduce_mean(w * keras.losses.binary_crossentropy(y_t, y_p))

    model.compile(keras.optimizers.Adam(cfg['lr']), loss=weighted_bce)
    return model

# ─────────────── Load Data ───────────────
print("\n📊 Loading data...")
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df.loc["2018-01-01":]
df = df.drop(columns=[c for c in DROP_COLS if c in df.columns])
df = df.dropna(subset=["target"]).dropna()

X, y = df.drop(columns='target').values, df['target'].astype(int).values
n_features = X.shape[1]
split = int(len(df) * (1 - VAL_FRAC))

scaler = StandardScaler().fit(X[:split])
X_tr_raw = scaler.transform(X[:split]).astype(np.float32)
X_va_raw = scaler.transform(X[split:]).astype(np.float32)
y_tr_raw, y_va_raw = y[:split], y[split:]

# Class weights
pos_rate = y_tr_raw.mean()
CW0 = np.float32(1.0)
CW1 = np.float32((1-pos_rate)/pos_rate if pos_rate else 1.0)

# ─────────────── Train ───────────────
X_tr, y_tr = make_windows(X_tr_raw, y_tr_raw, CFG['window'])
X_va, y_va = make_windows(X_va_raw, y_va_raw, CFG['window'])

model = build_tcn(CFG, n_features, CW0, CW1)

callbacks = [
    keras.callbacks.EarlyStopping(patience=EARLY_STOP, restore_best_weights=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.5, min_lr=1e-7, verbose=1)
]

start_time = datetime.now()
model.fit(X_tr, y_tr, validation_data=(X_va, y_va),
          epochs=EPOCHS, batch_size=CFG['batch'], shuffle=False,
          callbacks=callbacks, verbose=2)
train_time = datetime.now() - start_time

# ─────────────── Evaluate ───────────────
prob = model.predict(X_va, verbose=0).ravel()
pred = (prob >= 0.5).astype(int)

prec = precision_score(y_va, pred, zero_division=0)
rec = recall_score(y_va, pred, zero_division=0)
f1 = f1_score(y_va, pred, zero_division=0)
f1_w = weighted_f1(y_va, pred)
auc = roc_auc_score(y_va, prob)
acc = accuracy_score(y_va, pred)

print(f"\n📈 Results:")
print(f"   Precision:  {prec:.3f}")
print(f"   Recall:     {rec:.3f}")
print(f"   F1 Score:   {f1:.3f}")
print(f"   F1 (α=2):   {f1_w:.3f}")
print(f"   Accuracy:   {acc:.3f}")
print(f"   AUC:        {auc:.3f}")

# ─────────────── Save Predictions ───────────────
val_start_idx = split + CFG['window']
timestamps = df.index[val_start_idx:val_start_idx + len(prob)]

pred_df = pd.DataFrame({
    "timestamp": timestamps.strftime("%d/%m/%Y %H:%M"),
    "prob_up": prob,
    "prob_down": 1.0 - prob,
    "winning_prob": np.maximum(prob, 1.0 - prob),
    "prediction": pred,
    "actual": y_va
})

csv_name = f"{CFG['name']}_predictions.csv"
pred_df.to_csv(csv_name, index=False, float_format="%.6f")
print(f"✅ Predictions saved: {csv_name} ({len(pred_df):,} rows)")


🖥️ GPU Status:
   GPUs available: 0

📊 Loading data...
Epoch 1/60
396/396 - 44s - 111ms/step - loss: 0.8539 - val_loss: 0.6847 - learning_rate: 5.9500e-04
Epoch 2/60
396/396 - 39s - 98ms/step - loss: 0.6871 - val_loss: 0.6841 - learning_rate: 5.9500e-04
Epoch 3/60
396/396 - 28s - 71ms/step - loss: 0.6837 - val_loss: 0.6837 - learning_rate: 5.9500e-04
Epoch 4/60
396/396 - 28s - 72ms/step - loss: 0.6828 - val_loss: 0.6824 - learning_rate: 5.9500e-04
Epoch 5/60
396/396 - 28s - 72ms/step - loss: 0.6826 - val_loss: 0.6822 - learning_rate: 5.9500e-04
Epoch 6/60
396/396 - 29s - 72ms/step - loss: 0.6826 - val_loss: 0.6833 - learning_rate: 5.9500e-04
Epoch 7/60
396/396 - 41s - 103ms/step - loss: 0.6826 - val_loss: 0.6820 - learning_rate: 5.9500e-04
Epoch 8/60
396/396 - 28s - 72ms/step - loss: 0.6823 - val_loss: 0.6834 - learning_rate: 5.9500e-04
Epoch 9/60
396/396 - 28s - 71ms/step - loss: 0.6825 - val_loss: 0.6819 - learning_rate: 5.9500e-04
Epoch 10/60
396/396 - 28s - 72ms/step - loss: 0.6822

In [ ]:
"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\src\Models\models\models\TCN_HighestPrecision_predictions.csv"

In [7]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the predictions CSV
csv_path = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\src\Models\models\models\Trial_36_BestOverall_predictions.csv"
df = pd.read_csv(csv_path)

# Ensure column names are correct and lowercase
df.columns = df.columns.str.strip().str.lower()

# Extract actual and predicted values
y_true = df['actual']
y_pred = df['prediction']  # prediction at threshold 0.5

# Calculate metrics
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Print results
print("📊 Evaluation at threshold 0.5:")
print(f"Precision: {precision:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"F1 Score : {f1:.3f}")

📊 Evaluation at threshold 0.5:
Precision: 0.523
Recall   : 1.000
F1 Score : 0.686
